# MPModel: Infering the detergent corona around a membrane protein

Throughout this tutorial, we will buid a coarse-grained model of a micelle around the  membrane protein Aquaporin-0 (PDB: 2B6P [1]) and use the `simSPI` package to simulate a cryo-EM experiment of the structure. For that, it is necessary to have installed the TEM-simulator [2] and the python libraries: Matplotlib Numpy, Gemmi, Scikit-learn and Scipy.


<p align="center">
    <img src="figures/2b6p_map.png" alt="Drawing" style="width: 700px;"/>
    <p style="text-align: center">
        Figure 1: _Bos taurus_ Aquaporin-0 complex
    </p>
</p>


[1] https://www.rcsb.org/structure/2B6P

[2] http://tem-simulator.sourceforge.net/

In [1]:
import os
import sys
import warnings

sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import math
import gemmi
import torch
import mrcfile
from matplotlib import pyplot as plt, axes as ax
from scipy.spatial.transform import Rotation as rot

from simSPI.linear_simulator.linear_simulator import LinearSimulator
from test_linear_simulator import init_data

import detergent_belt as mpm
import atomic_models

from torchmetrics.image import UniversalImageQualityIndex

## Creating the model
### Extracting the protein atomic coordinates from a PDB file

In [3]:
ptn = mpm.MembraneProtein("6dmy.pdb") #path to pdb file

<p align="center">
    <img src="figures/aquaporin.png" alt="Drawing" style="width: 400px;"/>
    <p style="text-align: center">
        Figure 2: Representation of the atomic coordinates of aquaporin
    </p>
</p>

### Centering and rotating the structure
<p style='text-align: justify;margin-right:10%;'>
Using Principal Component Analysis (PCA), a method for data decomposition, we can extract the main axis where our data is distributed. We can use a matrix transformation as a function to represent a modification on our dataset. In this case, the transformation is aligning the principal component of the coordinate points to the Z-axis. Therefore, we can distribute the atoms on another axis and, by doing so, rotate the protein so that its transmembrane portion is positioned properly in space.
</p>

In [8]:
ptn.rotate_protein(axis='Z')

<p align="center">
    <img src="figures/aquaporin_rotated.png" alt="Drawing" style="width: 400px;"/>
    <p style="text-align: center">
        Figure 3: Aquaporin structure rotated and centered at the origin
    </p>
</p>

***
### Creating a corona around the protein

To create the Ellipsoid we will compute the points that satisfy the condition: 
$\frac{x^{2}}{a^{2}} + \frac{y^{2}}{b^{2}} + \frac{z^{2}}{c^{2}} >= 1$

Consider $O = (0,0,0)$, as the protein was centered after the coordinates were imported.

<p align="center">
    <img src="figures/ellipsoid.png" alt="Drawing" style="width: 200px;"/>
    <p style="text-align: center">
        Figure 4: The geometric parameters of an ellipsoid
    </p>
</p>

The method used in this program creates an ellipsoid by generating points iteratively in regular intervals dependent on the ray of the pseudoatom. After that, a function removes the coordinates in the most inner part of the ellipsoid that would conflict with the protein's atoms. This process' speed depends on the number of pseudo atoms, that itself depends on the parameters of the corona such as the height, determined by size of the transmembrane portion of the protein, and on the ray of the pseudo atoms filling the ellipsoid.

Let us create the hydrophobic interior ellipsoid of the corona with parameters chosen empirically for a better visualization of the exemple on Pymol.</br>

major axis &rarr; 35Å</br>
minor axis &rarr; 25Å</br>
ellipticity &rarr; 2</br>
pseudo atom type &rarr; CA</br>
pseudo atom ray &rarr; 1.5</br>

major axis &rarr; 40Å</br>
minor axis &rarr; 30Å</br>
ellipticity &rarr; 2</br>
pseudo atom type &rarr; N</br>
pseudo atom ray &rarr; 1.5

In [51]:
axis = 30
C = mpm.BeltCore()
C.set_belt_parameters(axis-10, 50, 16,[-2,-2,40])    #set the parameters for the core of your micelle
C.set_atomic_parameters(1.5, "CA")  #set the parameters for you hydrophilic pseudoatom
C.generate_ellipsoid()

S = mpm.BeltShell()
S.set_belt_parameters(axis, 60, 26,[-2,-2,40])    #set the parameters for the shell of your micelle
S.set_atomic_parameters(1.5, "N")   #set the parameters for you hydrophobic pseudoatom
S.generate_ellipsoid()

### Excluding the overlaping pseudoatoms
Now that we have our two ellipsoids we need to delete from our corona the pseudo atoms that are physically overlapping with other atoms
Observation: the next steps take a couple of minutes to run

In [52]:
C.in_hull(ptn.final_coordinates)
C.remove()
 
S.in_hull(C.coordinates_set)
S.remove()
S.in_hull(ptn.final_coordinates)      
S.remove()

### Generating the final PDB file

In [53]:
file_name = "outputs/pdb/6dmy/6dmy_30x60x26.pdb"   #name your pdb file
M = mpm.Model()
M.clean_gemmi_structure()
M.write_atomic_model(file_name, model=gemmi.Model("model"))
M.create_model(file_name,C,S,ptn)

<p align="center">
    <img src="figures/aqp_model_abc.png" alt="Drawing" style="width: 600px;"/>
    <p style="text-align: center">
        Figure 5: Model of detergent micelle and membrane protein Aquaporin (AQP0) seen as A- full model, B- micelle
hydrophobic core and C- frontal cut. The micelle hydrophobic core is represented in yellow,
the hydrophilic shell in blue and the protein in pink.
    </p>
</p>

XXXXXXXXXXXXX CHIMERAX XXXXXXXXXXXXX molmap #[structure] [res]

ADD STEPS
ADD IMAGE
MAYBE AUTOMIZE

## Linear Simulation

In [4]:
def rotation(N):
    A = np.random.uniform(0,2*np.pi,N)
    V = np.random.uniform(0,1,3)
    R = []
    for a in A:
        r = rot.from_rotvec(a*V)
        R.append(r.as_matrix())
    return R #sampled rotations

In [54]:
def LinSim(vol_path, data_path, rotations):
    '''
    rotations -> list of rotation matrices
    '''
    with mrcfile.open(vol_path) as file:
        vol = file.data
    saved_data, config = init_data(data_path)
    
    #nx = vol.shape[0]
    nx = 200
    config["input_volume_path"] = vol_path
    config["side_len"] = nx
    config["ctf_size"] = nx
    config["pixel_size"] = 1.2
    
    config["noise"] = True
    config["noise_sigma"] = 1

    lin_sim = LinearSimulator(config)
    
    N = len(rotations)
    
    ctf_params = saved_data["ctf_params"]
    shift_params = saved_data["shift_params"]
    rot_params = {'rotmat': torch.Tensor(rotations)}
    
    ctf_params["defocus_u"] = torch.tensor([1.1000,2.1000,1.5000,1.7000,1.1000,2.1000,1.5000,1.7000]).reshape(N,1,1,1)
    ctf_params["defocus_v"] = torch.tensor([1.0000,2.2000,1.6000,1.8000,1.10000,2.2000,1.6000,1.8000]).reshape(N,1,1,1)
    ctf_params["defocus_angle"] = torch.tensor([0.3142,1.5708,2.5133,3.1416,0.3142,1.5708,2.5133,3.1416]).reshape(N,1,1,1)
    shift_params["shift_x"] = torch.tensor([ 4.0000,  5.5000, -3.2000, -6.0000, 4.0000,  5.5000, -3.2000, -6.0000 ])
    shift_params["shift_y"] = torch.tensor([ 6.0000, -4.5000, -4.0000,  0.0000,  6.0000, -4.5000, -4.0000,  0.0000])

    particles = lin_sim(rot_params, ctf_params, shift_params)
    
    return particles #torch tensor for particle projection

In [24]:
#R = rotation(8)

f, ax = plt.subplots(2, 4, figsize=(12, 6))

#ax[0].imshow(proj[0, 0])
#ax[1].imshow(proj[1, 0])
#ax[2].imshow(proj[2, 0])
#ax[3].imshow(proj[3, 0])

ax[0,0].imshow(proj[0, 0])
ax[0,1].imshow(proj[1, 0])
ax[0,2].imshow(proj[2, 0])
ax[0,3].imshow(proj[3, 0])
ax[1,0].imshow(proj[4, 0])
ax[1,1].imshow(proj[5, 0])
ax[1,2].imshow(proj[6, 0])
ax[1,3].imshow(proj[7, 0])

### Simulate images

In [55]:
R = [[[ 0.80635711, -0.01370839,  0.59127007],
       [ 0.39410307,  0.75788644, -0.51989509],
       [-0.44098864,  0.65224245,  0.61652964]],
     [[ 0.06689542,  0.85905001,  0.50750181],
       [ 0.97395306, -0.16666956,  0.15374228],
       [ 0.21665741,  0.48399829, -0.84782382]],
     [[ 0.1111205 ,  0.99294078,  0.04148306],
       [ 0.75318596, -0.11137452,  0.64831059],
       [ 0.64835418, -0.04079614, -0.76024505]],
     [[ 0.92351078, -0.06264793,  0.37842182],
       [ 0.21290437,  0.9043648 , -0.36985948],
       [-0.31906044,  0.42213688,  0.84852866]],
    [[ 0.92061443, -0.12424634,  0.37017823],
       [ 0.23403454,  0.93444747, -0.26839478],
       [-0.31256504,  0.3337226 ,  0.88934376]],
    [[-0.23368781,  0.79206824,  0.56393077],
       [ 0.91409036, -0.01871608,  0.40507842],
       [ 0.33140433,  0.61014557, -0.71964822]],
    [[-0.1085439 ,  0.99362457, -0.03047033],
       [ 0.5394632 ,  0.08462134,  0.83774619],
       [ 0.83498364,  0.07449461, -0.54520902]], 
    [[ 0.39405398,  0.05288187,  0.91756469],
       [ 0.78512593,  0.49964088, -0.36597304],
       [-0.47780616,  0.86461697,  0.15536655]]]

In [60]:
sample = "6dmy_60x60x26"
root_dir = "/home/halv/"  # change this to your simSPI directory.
vol_path = f"{root_dir}compSPI_fork/simSPI/notebooks/outputs/mrc/6dmy/{sample}.mrc"
data_path = f"{root_dir}compSPI_fork/simSPI/tests/data/linear_simulator_data.npy"
proj_path = f"{root_dir}compSPI_fork/simSPI/notebooks/outputs/projections/6dmy/gt.npy"

In [61]:
proj = LinSim(vol_path, data_path, R)
torch.save(proj, proj_path)

In [ ]:
proj1 = torch.load(proj_path)
print(type(proj1))

f, ax = plt.subplots(2, 4, figsize=(12, 6))

ax[0,0].imshow(proj1[0, 0])
ax[0,1].imshow(proj1[1, 0])
ax[0,2].imshow(proj1[2, 0])
ax[0,3].imshow(proj1[3, 0])
ax[1,0].imshow(proj1[4, 0])
ax[1,1].imshow(proj1[5, 0])
ax[1,2].imshow(proj1[6, 0])
ax[1,3].imshow(proj1[7, 0])

<class 'torch.Tensor'>


## Parameter Inference

<p align="center">
    <img src="figures/parameters_new.png" alt="Drawing" style="width: 450px;"/>
    <p style="text-align: center">
        Figure 6: XXXXXXXXXXXXXXXXXXXXXXX ADD DESCRIPTION AND REFERENCE XXXXXXXXXXXXXXXXXXXX
    </p>
</p>

We will infer the parameter "a" by Rejection Approximation Bayesian Computation.

In [22]:
class ABC():
    '''
    e epsilon rejection criterion
    obs image vector of ground truth
    sim image vector of simulated model
    '''
    def l2(self, proj1, proj2):
        norm = []
        for i in range(len(proj1)):
            norm.append(torch.sum((proj1[i]-proj2[i])**2))
            rank = sorted(norm)
        return rank #np.mean(norm)

    def criterion(self, proj1, proj2):
        e = ((self.criterion*10000) + 40000)   #normalized criterion
        if l2(proj1,proj2) <= e:
            return True
            
    def generate_model():
        pass

    def runABC(self):
        samples = self.samples
        a = np.random.uniform()   #sample parameter a
        model = generate_model()  #create model using sampled a
        self.accepted = []
        for i in range(samples):   
            sim_proj = LinSim(vol_path, data_path, rot) #simulate image with generated model x100
            #superimpose images
            if criterion(obs_proj, sim_proj):
                self.accepted.append(a)   #save parameter

        self.result = np.mean(self.accepted)
        self.rank = sorted(self.accepted)

### Metrics

In [4]:
def mse(proj1, proj2):
    se = []
    for i in range(len(proj1)):
        se.append(torch.sum((proj1[i]-proj2[i])**2))
    return np.mean(se)
    
def rmsd(proj1, proj2):
    sd = []
    for i in range(len(proj1)):
        sd.append(torch.sum((proj1[i]-proj2[i])**2))
    return np.mean(np.sqrt(sd))

def psnr(proj1, proj2):
    p1 = torch.max(proj1)
    p2 = torch.max(proj2)
    if p1 >= p2:
        p = p1
    else:
        p = p2
    se = []
    for i in range(len(proj1)):
        se.append(torch.sum((proj1[i]-proj2[i])**2))
    mse = np.mean(se)
    psnr = (10*(np.log10(p)**2))/mse
    return psnr

In [3]:
from torchmetrics.image import MultiScaleStructuralSimilarityIndexMeasure as ms_ssim
from torchmetrics.image import UniversalImageQualityIndex

In [98]:
root_dir = "/home/halv/"  # change this to your simSPI directory.
gt = "6nsj_50x50x25"
gt_proj = torch.load(f"{root_dir}compSPI_fork/simSPI/notebooks/outputs/projections/6nsj/{gt}.npy")

dataset = dict()

In [114]:
sim = "6nsj_80x80x25"
sim_proj = torch.load(f"{root_dir}compSPI_fork/simSPI/notebooks/outputs/projections/6nsj/{sim}.npy")

In [115]:
uqi = UniversalImageQualityIndex()

distance = psnr(gt_proj, sim_proj)
dataset[sim] = distance
print(distance, dataset)

tensor(4.2166e-05) {'6nsj_45x45x25': tensor(7.1990e-05), '6nsj_50x50x25': tensor(0.0004), '6nsj_55x55x25': tensor(0.0001), '6nsj_60x60x25': tensor(5.9934e-05), '6nsj_65x65x25': tensor(4.3408e-05), '6nsj_70x70x25': tensor(3.9824e-05), '6nsj_75x75x25': tensor(3.9885e-05), '6nsj_80x80x25': tensor(4.2166e-05)}


### Simulated-based inference

In [3]:
def set_protein(path, ptn_axis):
    ptn = mpm.MembraneProtein(path)
    ptn.rotate_protein(ptn_axis)
    return ptn
        
def build_model(sample,ptn):
    micelle = mpm.DetergentBelt()
    micelle.set_belt_parameters(sample,50,25,7,[0,0,0]) #a1,a2,h,t,cset the parameters for the core of your micelle
    micelle.set_atomic_parameters(1.5, "CA", "N")  #set the parameters for hydrophobic and hydrophilic pseudoatoms
    micelle.generate_core()
    micelle.generate_shell()

    micelle.in_hull(micelle.core_coordinates_set,ptn.final_coordinates)
    micelle.remove(micelle.core_coordinates_set)

    micelle.in_hull(micelle.shell_coordinates_set,micelle.core_coordinates_set)
    micelle.remove(micelle.shell_coordinates_set)
    micelle.in_hull(micelle.shell_coordinates_set,ptn.final_coordinates)      
    micelle.remove(micelle.shell_coordinates_set)

    file_name = f"outputs/pdb/7wsn/7wsn_abc_{sample}x50x25.pdb"   #name your pdb file
    M = mpm.Model()
    M.clean_gemmi_structure()
    M.write_atomic_model(file_name, model=gemmi.Model("model"))
    M.create_model(file_name,micelle,ptn)

In [4]:
def ground_truth(ptn):
    sim_proj = LinSim(vol_path, data_path, rot) #simulate image with ground truth model
    return gt

def runABC(runs,ptn):
    for i in range(runs):
        a = np.random.uniform(30,35)   #sample parameter a
        model = build_model(a,ptn)  #create model using sampled a

        '''#accepted = []
        for i in range(samples):
            sim_proj = LinSim(vol_path, data_path, rot) #simulate image with generated model x100
            #superimpose images
            #if criterion(obs_proj, sim_proj):
            #    self.accepted.append(a)   #save parameter

            distance = uqi(gt_proj, sim_proj)
            dataset[sim] = distance

            result = np.mean(self.accepted)
            rank = sorted(self.accepted)'''
    print(a)

In [5]:
ptn = set_protein("7wsn.pdb", "Z")
uqi = UniversalImageQualityIndex()

#gt = ground_truth(ptn)
print(runABC(1,ptn))

test1
test2
test2
test2
test2
test2
test2
test2
test2
32.79703294382229
None
